In [14]:
import datetime
from fuzzywuzzy import fuzz
import pandas as pd
import re

In [15]:
substations = pd.read_csv('../data/California_Electric_Substations.csv',
                          usecols=['Name', 'ZIP_CODE', 'COUNTY', 'Lon', 'Lat'], dtype='str')
pge_substations = substations.sort_values('Name').reset_index(drop=True)

In [16]:
def clean_df_20_21(file_path, start_datetype, end_datetype):
    psps = pd.read_csv(file_path, dtype=str)
    if 'Distribution / Transmission' in psps.columns:
        psps['Distribution / Transmission'] = [
            x.upper() for x in psps['Distribution / Transmission']
        ]
        psps = psps[psps['Distribution / Transmission'] == 'DISTRIBUTION']
        psps.drop(columns='Distribution / Transmission', inplace=True)
    cleaned_circuit = [
        re.findall('.*?(?=\s\d{4,}\*?)', circuit)[0].strip()
        if len(re.findall('.*?(?=\s\d{4,}\*?)', circuit)) > 0 else circuit
        for circuit in psps['Circuit Name']
    ]
    psps['Circuit Name'] = cleaned_circuit
    cleaned_hftd = [
        int(max(re.findall('\d', hftd)))
        if len(re.findall('\d', hftd)) > 0 else 0
        for hftd in psps['HFTD Tier(s)']
    ]
    psps['HFTD Tier'] = cleaned_hftd
    psps.columns = [col_name.strip() for col_name in psps.columns]
    shorter_names = [
        re.sub(' Customers$', '', col_name) for col_name in psps.columns
    ]
    psps.columns = shorter_names
    psps.rename(columns={
        'De-Energization Date and Time (PDT)': 'DeEnergization Date and Time',
        'De-Energization Date and Time': 'DeEnergization Date and Time',
        'Restoration Date and Time (PDT)': 'Restoration Date and Time',
        'Counties': 'Key Communities',
        'Commercial/Industrial': 'Commercial / Industrial'
        },
        inplace=True
    )
    fixed_communities = [
        re.sub('[\r\n\s]+', ' ', comm) for comm in psps['Key Communities']
    ]
    def get_times(str_time, date_type):
        fixed_time = re.sub('[\r\n\s]+', ' ', str(str_time))
        try:
            time = pd.to_datetime(fixed_time, format=date_type)
        except:
            try:
                date_format = re.sub('y', 'Y', date_type)
                time = pd.to_datetime(fixed_time, format=date_format)
            except:
                time = pd.to_datetime('1970-01-01')
        return time
    start_time = [
        get_times(time, start_datetype)
        for time in psps['DeEnergization Date and Time']
    ]
    end_time = [
        get_times(time, end_datetype)
        for time in psps['Restoration Date and Time']
    ]
    psps['deenergize_time'] = start_time
    psps['restoration_time'] = end_time
    psps['time_out_min'] = (
        (psps.restoration_time - psps.deenergize_time) / pd.Timedelta('1m')
    )
    psps['Key Communities'] = fixed_communities
    for col in ['HFTD Tier', 'Total', 'Residential', 'Commercial / Industrial',
        'Medical Baseline', 'Other']:
        psps[col] = psps[col].astype(int)
    psps = psps[[    
        'Circuit Name', 'deenergize_time', 'restoration_time', 'time_out_min',
        'Key Communities', 'HFTD Tier', 'Total', 'Residential'
    ]]
    psps.columns = [
        'circuit_name', 'deenergize_time', 'restoration_time', 'time_out_min', 'key_communities', 'hftd_tier', 'total_affected', 'residential_affected'
    ]
    return psps

In [17]:
file_names = [
    '../data/PSPS-{}-circuits.csv'.format(dt)
    for dt in [
        '09.07.20', '09.27.20', '10.14.20', '10.21.20', '10.25.20', '12.02.20',
        '01.19.21', '08.17.21', '09.20.21', '10.11.21', '10.14.21'
    ]
]
start_formats = [
    '%m/%d/%y %H:%M', '%m/%d/%Y %H:%M', '%m/%d/%Y %H:%M', '%d/%m/%y %H:%M',
    '%m/%d/%Y %H:%M', '%Y-%m-%d %H:%M:%S', '%m/%d/%Y %H:%M', '%d/%m/%y %H:%M',
    '%d/%m/%y %H:%M', '%d/%m/%y %H:%M', '%d/%m/%y %H:%M'
]
end_formats = [
    '%m/%d/%y %H:%M', '%m/%d/%Y %H:%M', '%m/%d/%Y %H:%M', '%Y-%m-%d %H:%M:%S',
    '%m/%d/%Y %H:%M', '%Y-%m-%d %H:%M:%S', '%m/%d/%Y %H:%M', '%d/%m/%y %H:%M',
    '%d/%m/%y %H:%M', '%d/%m/%y %H:%M', '%d/%m/%y %H:%M'
]
data_input = [(file_names[i], start_formats[i], end_formats[i]) for i in range(len(file_names))]

data = pd.concat(
    [clean_df_20_21(file[0], file[1], file[2]) for file in data_input],
    axis=0
).reset_index(drop=True)

In [18]:
data = data[(data.restoration_time > '2000-01-01') &
            (data.deenergize_time > '2000-01-01')].reset_index(drop=True)

In [19]:
data

,circuit_name,og_deeng_time,deenergize_time,og_restore_time,restoration_time,time_out_min,key_communities,hftd_tier,total_affected,residential_affected
0,ALLEGHANY,9/7/20 21:57,2020-09-07 21:57:00,9/10/20 16:31,2020-09-10 16:31:00,3994.0,"ALLEGHANY, DOWNIEVILLE, GOODYEARS BAR, SIERRA ...",3,1028,865
1,ALLEGHANY,9/7/20 21:57,2020-09-07 21:57:00,9/9/20 15:28,2020-09-09 15:28:00,2491.0,"ALLEGHANY, WASHINGTON",3,151,127
2,ALPINE,9/7/20 23:03,2020-09-07 23:03:00,9/9/20 17:34,2020-09-09 17:34:00,2551.0,"ANGELS CAMP, BEAR VALLEY",0,276,271
3,ALPINE,9/7/20 23:03,2020-09-07 23:03:00,9/9/20 17:33,2020-09-09 17:33:00,2550.0,"ANGELS CAMP, BEAR VALLEY",0,303,269
4,ANTLER,9/8/20 01:08,2020-09-08 01:08:00,9/9/20 10:08,2020-09-09 10:08:00,1980.0,LAKEHEAD,3,913,771
...,...,...,...,...,...,...,...,...,...,...
905,LAMONT,15/10/21 01:09,2021-10-15 01:09:00,15/10/21 15:10,2021-10-15 15:10:00,841.0,KERN,2,2,0
906,MAGUNDEN,15/10/21 01:05,2021-10-15 01:05:00,15/10/21 15:12,2021-10-15 15:12:00,847.0,KERN,2,38,20
907,SCE TEHACHAPI,15/10/21 01:16,2021-10-15 01:16:00,15/10/21 14:26,2021-10-15 14:26:00,790.0,KERN,2,3,2
908,TEJON,15/10/21 01:07,2021-10-15 01:07:00,16/10/21 16:32,2021-10-16 16:32:00,2365.0,KERN,2,595,479


In [20]:
def most_similar_station(circuit, stns, thresh=80):
    sim_scores =[
        (
            fuzz.token_sort_ratio(circuit, stn.Name),
            stn.Name, stn.ZIP_CODE, stn.Lon, stn.Lat
        )
        for _, stn in stns.iterrows()
    ]
    max_score = max([scores[0] for scores in sim_scores])
    if max_score < thresh:
        return ('Default', None, None, None)
    return [scores[1:] for scores in sim_scores if scores[0] == max_score][0]

In [21]:
unique_circuits = data['circuit_name'].unique()
closest_substation = {
    circuit: most_similar_station(circuit, pge_substations)
    for circuit in unique_circuits
}

In [22]:
zips = [closest_substation[circuit][1] for circuit in data['circuit_name']]
longs = [closest_substation[circuit][2] for circuit in data['circuit_name']]
lats = [closest_substation[circuit][3] for circuit in data['circuit_name']]
data['zip_code'] = zips
data['longitude'] = longs
data['latitude'] = lats
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   circuit_name          910 non-null    object        
 1   og_deeng_time         910 non-null    object        
 2   deenergize_time       910 non-null    datetime64[ns]
 3   og_restore_time       910 non-null    object        
 4   restoration_time      910 non-null    datetime64[ns]
 5   time_out_min          910 non-null    float64       
 6   key_communities       910 non-null    object        
 7   hftd_tier             910 non-null    int64         
 8   total_affected        910 non-null    int64         
 9   residential_affected  910 non-null    int64         
 10  zip_code              866 non-null    object        
 11  longitude             866 non-null    object        
 12  latitude              866 non-null    object        
dtypes: datetime64[ns](2)

In [24]:
data[data.zip_code.isnull()].to_csv('missing_zips.csv')

In [ ]:
census_data = pd.read_csv('../data/ACSDP5Y2020.DP05-Data.csv')
census_data = census_data[
    [column for column in census_data.columns if re.search('E$', column)]
]
census_pop = census_data[['NAME', 'DP05_0001E']] # ZCTA, total population cols
census_pop.columns == ['name', 'total_pop']
census_pop.drop(index=0, inplace=True)
census_pop['ZCTA'] = [re.findall('\d{5}', obs)[0] for obs in census_pop.NAME]

zip_zcta = pd.read_excel(
    '../data/zip-code-zcta.xlsx', dtype='str'
)[['ZIP_CODE', 'ZCTA']]

# Join the two data sets
zip_census = pd.merge(census_pop, zip_zcta, how='inner', on='ZCTA')
zip_census.DP05_0001E = zip_census.DP05_0001E.astype(float)

In [ ]:
zip_census.dtypes